# Titanic med CatBoost

## Frågeställningar
1) Vad för typ av boosting är det?
2) Behövs encoding för kategoriska värden?
3) Kan den hantera missing values? (NaN/None etc)
4) Hur lång tid tar det att träna en modell
5) Hur många hyperparametrar finns det och vilka är de viktigaste
6) Hur bra fungerar den på större dataset?

#### Importerar bibliotek